In [146]:
import pandas as pd
import json

all_labels = pd.read_csv("./datasets/ePillID_data/all_labels.csv")
with open('./drug/ndc/drug-ndc-0001-of-0001.json', 'r') as file:
    data = json.loads(file.read())
file.close()
df = pd.DataFrame(data['results'])

#filter out anything that isn't a tablet or capsule
pills = df[df['dosage_form'].str.contains('TABLET') | df['dosage_form'].str.contains('CAPSULE')]

#filter out anything that isn't for humans
pills = pills[pills['product_type'].str.contains('HUMAN')]

#pillbox retired on date Jan. 29 2021 as per https://www.nlm.nih.gov/pubs/techbull/ja20/ja20_pillbox_discontinue.html
pillbox_retirement = 20210129
x=pills["product_ndc"].str.split("-", expand=True).astype(int).rename(columns={0:"label_code_id", 1: "prod_code_id"}).join(pills["marketing_start_date"])
merged = pd.merge(x, all_labels.get(["label_code_id", "prod_code_id"]), on=["label_code_id", "prod_code_id"], how="outer", indicator=True)
ndc_not_in_dataset = merged[merged['_merge'] == "left_only"].drop_duplicates()


In [ ]:
all_labels.get(["label_code_id", "prod_code_id"]).drop_duplicates()
all_labels["label"].drop_duplicates()

0                                   51285-0092-87_BE305F72
1                                   00093-0148-01_4629A34D
2                                   00093-7248-06_7829BC3D
3                                   00093-0928-06_6926B4E5
4                                   50111-0459-01_1C300E70
                               ...                        
13522    1ee933c3c14cc5f987d88b9bf853b8d932bf7efb24cc2c...
13524    8682b81628ffb6203e45ffabbf6984fb85b47d65ee1045...
13526    cd1524c3bce8a11f19e3b8cbc1b5391becd9a3c6833ff8...
13528    6234bd92e8013b2eb1d2ead52a1034250c08228d194137...
13530    bb28d4d9053e8f4e9cfea5df555bb90670a14d7f7f5bb9...
Name: label, Length: 4902, dtype: object

In [129]:
merged[merged['_merge'] == "left_only"].drop_duplicates()

,label_code_id,prod_code_id,marketing_start_date,_merge
0,2,2980,20200508,left_only
1,2,3227,20021126,left_only
9,2,3230,20070409,left_only
10,2,3231,20031224,left_only
11,2,3235,20040824,left_only
...,...,...,...,...
58874,90096,133,20220205,left_only
58875,90096,134,20220205,left_only
58876,90096,135,20220205,left_only
58877,90096,141,20221203,left_only


In [132]:
merged[merged['_merge'] == "both"].drop_duplicates()

,label_code_id,prod_code_id,marketing_start_date,_merge
2,2,3228,20021126,both
5,2,3229,20021126,both
12,2,3238,20021126,both
16,2,3239,20021126,both
21,2,3250,20050214,both
...,...,...,...,...
45164,69238,1251,20141001,both
45381,69315,136,20121128,both
55756,75826,114,20151223,both
55951,75987,10,20111205,both


In [38]:
merged[merged['_merge'] == "right_only"].drop_duplicates()

,label_code_id,prod_code_id,marketing_start_date,_merge
64,2,5123,NaN,right_only
94,3,4222,NaN,right_only
155,6,717,NaN,right_only
157,6,747,NaN,right_only
159,6,749,NaN,right_only
...,...,...,...,...
56710,76519,1004,NaN,right_only
58878,99207,461,NaN,right_only
58882,99207,463,NaN,right_only
58884,99207,465,NaN,right_only


In [42]:
from download import downloadZip

downloadZip("https://www.accessdata.fda.gov/cder/ndc_excluded.zip", "./excluded_drugs/")

In [47]:
import chardet

with open("/Users/Amanda/Desktop/PillRecognition/excluded_drugs/Products_excluded.xls", "rb") as file:
    print(chardet.detect(file.read()))
file.close()

{'encoding': 'Windows-1252', 'confidence': 0.7299955242656871, 'language': ''}


In [ ]:
excluded = pd.read_table('/Users/Amanda/Desktop/PillRecognition/excluded_drugs/Products_excluded.xls', encoding='Windows-1252')
excluded_ndcs = excluded["PRODUCTNDC"]
excluded_ndcs = excluded_ndcs[excluded_ndcs.str.contains('^(\d+)-(\d+)$', regex=True)].str.split("-", expand=True).astype(int).rename(columns={0:"label_code_id", 1: "prod_code_id"})

ex = pd.merge(merged[merged['_merge'] == "right_only"].drop_duplicates().get(["label_code_id", "prod_code_id"]), excluded_ndcs, on=["label_code_id", "prod_code_id"], how="outer", indicator=True)
ex[ex["_merge"] == "left_only"]

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.


,label_code_id,prod_code_id,_merge
4191,149,7,left_only
4534,173,758,left_only
4535,173,759,left_only
4536,173,760,left_only
11629,555,9008,left_only
11630,555,9010,left_only
11632,555,9012,left_only
11634,555,9014,left_only
11635,555,9016,left_only
11637,555,9026,left_only
